# Notebook to play with paper reviews

In [1]:
import sys
from pycomfort.files import *
from pathlib import Path
import polars as pl
from functional import seq
import string
import pprint
import time
pp = pprint.PrettyPrinter(indent=4)

local = (Path("..") / "biotables").resolve()
if local.exists():
    sys.path.insert(0, Path("..").absolute().as_posix())
    sys.path.insert(0, local)
    print(sys.path)
    %load_ext autoreload
    %autoreload 2

[PosixPath('/home/antonkulaga/sources/biotables/biotables'), '/home/antonkulaga/sources/biotables/notebooks/..', '/home/antonkulaga/sources/biotables/notebooks', '/home/antonkulaga/sources/biotables', '/home/antonkulaga/micromamba/envs/biotables/lib/python310.zip', '/home/antonkulaga/micromamba/envs/biotables/lib/python3.10', '/home/antonkulaga/micromamba/envs/biotables/lib/python3.10/lib-dynload', '', '/home/antonkulaga/.local/lib/python3.10/site-packages', '/home/antonkulaga/micromamba/envs/biotables/lib/python3.10/site-packages']


In [2]:
base = Path("..").resolve()
base = base.parent if "notebooks" in base.name else base
from biotables.locations import Locations
locations = Locations(base)
locations.papers


PosixPath('/home/antonkulaga/sources/biotables/data/papers')

In [3]:
from getpaper.parse import download_and_parse, PDFParser

doi = "10.1038/s41586-020-2975-4"
folders = download_and_parse(doi, locations.papers, scihub_on_fail=True, subfolder=True, do_not_reparse=True, parser=PDFParser.py_mu_pdf)
folders

[PosixPath('/home/antonkulaga/sources/biotables/data/papers/10.1038/s41586-020-2975-4')]

In [4]:
def ask_paper(paper: AskPaper):
    folders: List[Path] = download_and_parse(paper.doi, subfolder=True, do_not_reparse=True)
    if len(folders) <1:
        loguru.logger.error(f"nothing was downloaded/parsed for {paper.doi}")
    loguru.logger.info(f"download folder {paper.doi}")
    return folders

[PosixPath('/home/antonkulaga/sources/biotables/data/papers/10.1038/s41586-020-2975-4')]

In [38]:

from pydantic import BaseModel
from getpaper.download import try_download

class AskPaper(BaseModel):
    doi: str
    prompt: Optional[str] = None


def ask_paper(paper: AskPaper):
    doi = paper.doi
    d: Try[(str, Path, Path)] = try_download(doi, locations.papers, True)
    """
    Success(_value=('10.1038/s41586-020-2975-4', PosixPath('/home/antonkulaga/sources/biotables/data/papers/10.1038/s41586-020-2975-4.pdf'), PosixPath('/home/antonkulaga/sources/biotables/data/papers/10.1038/s41586-020-2975-4_meta.json')))
    """
    return d.map(lambda d: d[1]).get_or_else_get(lambda e: None)
   
    
r = ask_paper(AskPaper(doi = "10.1038/s41586-020-2975-4"))

[INFO] | 2023/11/15 03:32:34 | Choose scihub url [0]: https://sci-hub.ru
[INFO] | 2023/11/15 03:32:34 | <- Request: scihub_url=https://sci-hub.ru, source=DoiSource[type=doi, id=doi.org/10.1038/s41586-020-2975-4], proxies={}
[INFO] | 2023/11/15 03:32:34 | -> Response: status_code=200, content_length=7035
[INFO] | 2023/11/15 03:32:34 | * Extracted information: {'url': 'https://sci-hub.ru/downloads/2020-12-02/1e/10.1038@s41586-020-2975-4.pdf', 'title': 'Reprogramming to recover youthful epigenetic information and restore vision. Nature, 588(7836), 124–129'}


 10% [=====                                             ] 12279808/112223116

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



 27% [=============                                     ] 30474240/112223116

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



 62% [===============================                   ] 70246400/112223116

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



 78% [=======================================           ] 88600576/112223116

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
from getpaper.parse import download_and_parse_command

In [42]:
doi = "10.1038/s41586-020-2975-4"
from getpaper.parse import download_and_parse
folders = download_and_parse(doi, locations.papers)
folders

[]

## Open the spreadsheets

In [12]:
from biotables.biotable import BioTable
table: BioTable = BioTable("https://docs.google.com/spreadsheets/d/1VZvyYw78uTWf95xMr4z2UTBNN71KAUO6mkwzoAyb03A")

/tmp/ipykernel_155142/1022393964.py:2: DeprecationWarning: [Deprecated][in version 6.0.0]: client_factory will be replaced by gspread.http_client types
  table: BioTable = BioTable("https://docs.google.com/spreadsheets/d/1VZvyYw78uTWf95xMr4z2UTBNN71KAUO6mkwzoAyb03A")


In [46]:
from getpaper.download import *
#schihub_doi("10.1038/s43587-022-00183-2", table.papers)
try_download("10.7554/eLife.71624", table.papers)

[INFO] | 2023/10/24 00:52:04 | Choose scihub url [0]: https://sci-hub.ru
[INFO] | 2023/10/24 00:52:04 | <- Request: scihub_url=https://sci-hub.ru, source=DoiSource[type=doi, id=doi.org/10.7554/eLife.71624], proxies={}
[INFO] | 2023/10/24 00:52:04 | -> Response: status_code=200, content_length=5828
[WARNING] | 2023/10/24 00:52:04 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2023/10/24 00:52:04 | Choose scihub url [1]: http://sci-hub.st
[INFO] | 2023/10/24 00:52:04 | <- Request: scihub_url=http://sci-hub.st, source=DoiSource[type=doi, id=doi.org/10.7554/eLife.71624], proxies={}
[INFO] | 2023/10/24 00:52:05 | -> Response: status_code=200, content_length=5828
[WARNING] | 2023/10/24 00:52:05 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2023/10/24 00:52:05 | Choose scihub url [2]: https://sci-hub.se
[INFO] | 2023/10/24 00:52

Success(_value=('10.7554/eLife.71624', PosixPath('/home/antonkulaga/sources/biotables/data/papers/10.7554/eLife.71624.pdf'), PosixPath('/home/antonkulaga/sources/biotables/data/papers/10.7554/eLife.71624_meta.json')))

In [13]:
table.download_papers()

index is 2


[INFO] | 2023/10/24 00:21:15 | Choose scihub url [0]: https://sci-hub.ru
[INFO] | 2023/10/24 00:21:15 | <- Request: scihub_url=https://sci-hub.ru, source=DoiSource[type=doi, id=doi.org/10.1016/j.stemcr.2022.09.009], proxies={}
[INFO] | 2023/10/24 00:21:15 | -> Response: status_code=200, content_length=5837
[WARNING] | 2023/10/24 00:21:15 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2023/10/24 00:21:15 | Choose scihub url [1]: http://sci-hub.st
[INFO] | 2023/10/24 00:21:15 | <- Request: scihub_url=http://sci-hub.st, source=DoiSource[type=doi, id=doi.org/10.1016/j.stemcr.2022.09.009], proxies={}
[INFO] | 2023/10/24 00:21:16 | -> Response: status_code=200, content_length=5837
[WARNING] | 2023/10/24 00:21:16 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2023/10/24 00:21:16 | Choose scihub url [2]: https://sci-hub.se
[INFO] 

TypeError: Failure.get_or_else_get() missing 1 required positional argument: 'default'

In [ ]:
doi_ids = table_1.col_values(1)

from getpaper import download, parse

bio = papers.BioTable(locations.papers)

# Iterate over the DOI IDs, get the paper details/text, and update the neighboring column (column B in this case)
for index, doi_id in enumerate(doi_ids[ids_start-1:], start=ids_start):  # using start=1 because spreadsheet indexing starts from 1
    paper_details = bio.from_doi(doi_id)
    table_1.update_cell(index, 2, paper_details)  # 2 refers to column B